In [ ]:
# Bayesian Box APM: https://www.intraocular.net/posts/bayesian-box-plus-minus
# RAPM: https://squared2020.com/2017/09/18/deep-dive-on-regularized-adjusted-plus-minus-i-introductory-example/
# Really interesting NBA data scraping example repository: https://github.com/jecutter/nba-data-models
# Sample play-by-play data scrape: https://jman4190.medium.com/how-to-accessing-live-nba-play-by-play-data-f24e02b0a976

In [101]:
import requests
import json
import pandas as pd
import numpy as np
# from bs4 import BeautifulSoup
# import re
from sklearn.preprocessing import MultiLabelBinarizer
# from bs4 import BeautifulSoup
# from sklearn.model_selection import train_test_split
# from sklearn import tree
# from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix
# import matplotlib.pyplot as plt
# from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge, LinearRegression
# import tensorflow as tf
pd.set_option('display.max_columns',500)

In [34]:
# Headers for querying NBA JSON data api
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [41]:
lineup_url = 'https://stats.nba.com/stats/leaguedashlineups?Conference=&DateFrom=&DateTo=&Division=&GameID=&GameSegment=&GroupQuantity=5&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlusMinus=N&Rank=N&Season={}&SeasonSegment=&SeasonType={}&ShotClockRange=&TeamID=0&VsConference=&VsDivision='

In [57]:
season = '2021-22'
season_type = 'Regular+Season'
lineup_url = lineup_url = 'https://stats.nba.com/stats/leaguedashlineups?\
Conference=&DateFrom=&DateTo=&Division=&GameID=&GameSegment=&GroupQuantity=5\
&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0\
&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlusMinus=N&Rank=N\
&Season={}&SeasonSegment=&SeasonType={}&ShotClockRange=&TeamID=0\
&VsConference=&VsDivision='.format(season,season_type)

In [58]:
r = requests.get(url=lineup_url, headers=headers, params=params).json()
df = pd.DataFrame(r['resultSets'][0]['rowSet'],columns=r['resultSets'][0]['headers'])
df.sort_values(by='NET_RATING',ascending=False).head()

,GROUP_SET,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,OFF_RATING_RANK,DEF_RATING_RANK,NET_RATING_RANK,AST_PCT_RANK,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
1985,Lineups,-1629628-1629629-1630193-1630540-1630579-,R. Barrett - C. Reddish - I. Quickley - M. McB...,1610612752,NYK,3,2,1,0.667,12.0,163.2,158.3,63.1,68.0,100.2,90.3,0.417,5.00,16.7,0.167,0.800,0.654,0.042,0.750,0.816,97.0,94.58,78.82,24,1.063,2316,1646,5744,6286,1907,2000,4415,1687,10301,261,6929,8669,6350,3520,7326,2453,2426,13947,1696
1729,Lineups,-201152-1628449-1629018-1630173-1630567-,T. Young - C. Boucher - G. Trent Jr. - P. Achi...,1610612761,TOR,4,3,1,0.750,14.0,143.5,148.1,69.9,66.7,73.6,81.5,0.625,10.00,24.4,0.333,0.500,0.423,0.037,0.696,0.693,91.1,91.70,76.42,27,0.857,1571,956,5744,5984,1643,2795,3759,1899,6746,17,3230,4591,11422,10904,7301,3551,4315,14892,3026
1155,Lineups,-201959-202692-203095-203944-1630193-,T. Gibson - A. Burks - E. Fournier - J. Randle...,1610612752,NYK,7,3,4,0.429,19.0,126.9,135.1,61.3,54.1,65.6,81.1,0.471,2.00,14.5,0.455,0.909,0.682,0.108,0.553,0.590,93.4,95.97,79.98,37,0.971,612,956,16599,9025,1160,3547,3265,1901,10031,2231,8498,3404,5280,2486,9162,6601,6853,13477,2575
1623,Lineups,-2730-201566-201961-203076-1628370-,D. Howard - R. Westbrook - W. Ellington - A. D...,1610612747,LAL,3,2,1,0.667,14.0,152.9,164.3,92.4,88.9,60.5,75.4,0.421,8.00,20.0,0.444,0.900,0.684,0.036,0.778,0.765,95.1,93.30,77.75,28,0.869,2316,1646,5744,6286,1641,1771,6263,2023,10294,45,4617,3439,5323,2472,7287,2365,2929,14394,2990
1940,Lineups,-202066-203503-1626195-1629109-1629607-,G. Temple - T. Snell - W. Hernangomez - G. Cla...,1610612740,NOP,2,1,1,0.500,12.0,133.5,144.0,71.1,69.2,62.4,74.8,0.583,1.75,17.5,0.615,0.714,0.667,0.160,0.540,0.622,103.8,101.30,84.41,25,0.892,3707,3401,5744,7220,1979,2925,4444,2120,7466,3492,6725,1448,8390,2506,11527,6864,5970,10739,2871


In [111]:
test_df = df[(df['TEAM_ABBREVIATION']=='DAL')&(df['MIN']>=20)&(df['GP']>=10)]
test_df['GROUP_LIST'] = test_df['GROUP_NAME'].str.split(' - ')
test_df.head()

C:\Users\jwesi\AppData\Local\Temp\ipykernel_25680\1200353479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['GROUP_LIST'] = test_df['GROUP_NAME'].str.split(' - ')


,GROUP_SET,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,OFF_RATING_RANK,DEF_RATING_RANK,NET_RATING_RANK,AST_PCT_RANK,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK,GROUP_LIST
10,Lineups,-203493-203939-1627827-1628973-1629029-,R. Bullock - D. Powell - D. Finney-Smith - J. ...,1610612742,DAL,40,30,10,0.750,367.0,126.9,125.9,111.5,111.8,15.4,14.1,0.564,2.09,18.8,0.234,0.733,0.503,0.122,0.631,0.658,94.2,94.48,78.73,723,0.560,4,2,17335,5984,11,4585,10197,6215,7742,2214,5934,7613,8208,6612,9747,4705,5082,13972,6771,"[R. Bullock, D. Powell, D. Finney-Smith, J. Br..."
56,Lineups,-203939-1627827-1628467-1628973-1629029-,D. Powell - D. Finney-Smith - M. Kleber - J. B...,1610612742,DAL,20,14,6,0.700,142.0,105.5,105.3,117.8,117.7,-12.4,-12.4,0.600,1.43,17.3,0.256,0.715,0.483,0.163,0.536,0.570,94.4,94.54,78.78,282,0.459,46,27,17121,6259,57,7259,10969,10166,6993,4254,6805,6691,8389,9363,11576,6943,7444,13956,8901,"[D. Powell, D. Finney-Smith, M. Kleber, J. Bru..."
91,Lineups,-203501-203939-204001-1627827-1629029-,T. Hardaway Jr. - D. Powell - K. Porzingis - D...,1610612742,DAL,11,7,4,0.636,103.0,97.4,95.8,114.1,113.0,-16.7,-17.1,0.636,1.96,16.7,0.240,0.839,0.490,0.116,0.456,0.487,99.3,100.63,83.86,216,0.416,228,212,16599,6892,92,9682,10341,10673,6644,3357,6929,7560,5893,9262,9583,9717,10517,11066,9751,"[T. Hardaway Jr., D. Powell, K. Porzingis, D. ..."
93,Lineups,-203493-1627827-1628467-1628973-1629029-,R. Bullock - D. Finney-Smith - M. Kleber - J. ...,1610612742,DAL,19,10,9,0.526,101.0,105.9,105.8,104.3,103.4,1.6,2.4,0.513,1.58,14.2,0.357,0.684,0.520,0.125,0.492,0.530,97.4,97.88,81.56,208,0.514,55,78,17307,7216,94,7207,9329,7219,8086,3793,9192,4510,8927,6433,9775,9303,8581,12537,7704,"[R. Bullock, D. Finney-Smith, M. Kleber, J. Br..."
102,Lineups,-203501-203939-1627827-1628973-1629029-,T. Hardaway Jr. - D. Powell - D. Finney-Smith ...,1610612742,DAL,19,13,6,0.684,95.0,117.3,119.6,110.0,107.3,7.3,12.3,0.477,1.52,15.4,0.273,0.773,0.523,0.132,0.574,0.619,103.2,103.45,86.21,204,0.531,55,38,17121,6282,103,5512,9666,6368,10015,3834,8282,6484,7112,6361,10334,6068,6023,9597,7382,"[T. Hardaway Jr., D. Powell, D. Finney-Smith, ..."


In [112]:
mlb = MultiLabelBinarizer()
test_df = test_df.merge(pd.DataFrame(mlb.fit_transform(test_df['GROUP_LIST']),columns=mlb.classes_, index=test_df.index),
                     how='inner', left_index=True, right_index=True).drop(['GROUP_LIST'],axis=1)
players = list(mlb.classes_)
y_values = ['OFF_RATING','DEF_RATING','NET_RATING',
                          'E_OFF_RATING','E_DEF_RATING','E_NET_RATING']
test_df.head()

,GROUP_SET,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,OFF_RATING_RANK,DEF_RATING_RANK,NET_RATING_RANK,AST_PCT_RANK,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK,D. Bertans,D. Finney-Smith,D. Powell,J. Brunson,J. Green,K. Porzingis,L. Doncic,M. Kleber,R. Bullock,S. Dinwiddie,T. Hardaway Jr.
10,Lineups,-203493-203939-1627827-1628973-1629029-,R. Bullock - D. Powell - D. Finney-Smith - J. ...,1610612742,DAL,40,30,10,0.750,367.0,126.9,125.9,111.5,111.8,15.4,14.1,0.564,2.09,18.8,0.234,0.733,0.503,0.122,0.631,0.658,94.2,94.48,78.73,723,0.560,4,2,17335,5984,11,4585,10197,6215,7742,2214,5934,7613,8208,6612,9747,4705,5082,13972,6771,0,1,1,1,0,0,1,0,1,0,0
56,Lineups,-203939-1627827-1628467-1628973-1629029-,D. Powell - D. Finney-Smith - M. Kleber - J. B...,1610612742,DAL,20,14,6,0.700,142.0,105.5,105.3,117.8,117.7,-12.4,-12.4,0.600,1.43,17.3,0.256,0.715,0.483,0.163,0.536,0.570,94.4,94.54,78.78,282,0.459,46,27,17121,6259,57,7259,10969,10166,6993,4254,6805,6691,8389,9363,11576,6943,7444,13956,8901,0,1,1,1,0,0,1,1,0,0,0
91,Lineups,-203501-203939-204001-1627827-1629029-,T. Hardaway Jr. - D. Powell - K. Porzingis - D...,1610612742,DAL,11,7,4,0.636,103.0,97.4,95.8,114.1,113.0,-16.7,-17.1,0.636,1.96,16.7,0.240,0.839,0.490,0.116,0.456,0.487,99.3,100.63,83.86,216,0.416,228,212,16599,6892,92,9682,10341,10673,6644,3357,6929,7560,5893,9262,9583,9717,10517,11066,9751,0,1,1,0,0,1,1,0,0,0,1
93,Lineups,-203493-1627827-1628467-1628973-1629029-,R. Bullock - D. Finney-Smith - M. Kleber - J. ...,1610612742,DAL,19,10,9,0.526,101.0,105.9,105.8,104.3,103.4,1.6,2.4,0.513,1.58,14.2,0.357,0.684,0.520,0.125,0.492,0.530,97.4,97.88,81.56,208,0.514,55,78,17307,7216,94,7207,9329,7219,8086,3793,9192,4510,8927,6433,9775,9303,8581,12537,7704,0,1,0,1,0,0,1,1,1,0,0
102,Lineups,-203501-203939-1627827-1628973-1629029-,T. Hardaway Jr. - D. Powell - D. Finney-Smith ...,1610612742,DAL,19,13,6,0.684,95.0,117.3,119.6,110.0,107.3,7.3,12.3,0.477,1.52,15.4,0.273,0.773,0.523,0.132,0.574,0.619,103.2,103.45,86.21,204,0.531,55,38,17121,6282,103,5512,9666,6368,10015,3834,8282,6484,7112,6361,10334,6068,6023,9597,7382,0,1,1,1,0,0,1,0,0,0,1


In [113]:
test_df_regress = test_df[players+y_values]
test_df_regress

,D. Bertans,D. Finney-Smith,D. Powell,J. Brunson,J. Green,K. Porzingis,L. Doncic,M. Kleber,R. Bullock,S. Dinwiddie,T. Hardaway Jr.,OFF_RATING,DEF_RATING,NET_RATING,E_OFF_RATING,E_DEF_RATING,E_NET_RATING
10,0,1,1,1,0,0,1,0,1,0,0,125.9,111.8,14.1,126.9,111.5,15.4
56,0,1,1,1,0,0,1,1,0,0,0,105.3,117.7,-12.4,105.5,117.8,-12.4
91,0,1,1,0,0,1,1,0,0,0,1,95.8,113.0,-17.1,97.4,114.1,-16.7
93,0,1,0,1,0,0,1,1,1,0,0,105.8,103.4,2.4,105.9,104.3,1.6
102,0,1,1,1,0,0,1,0,0,0,1,119.6,107.3,12.3,117.3,110.0,7.3
241,0,1,1,0,0,0,1,0,1,0,1,77.0,105.1,-28.1,78.7,109.0,-30.3
283,0,1,0,1,0,1,1,0,0,0,1,92.5,109.6,-17.1,98.6,111.8,-13.1
465,0,1,1,0,0,0,1,1,1,0,0,105.6,108.0,-2.4,107.6,111.1,-3.5
491,0,1,1,0,0,0,1,0,1,1,0,117.9,123.1,-5.2,116.5,120.1,-3.6
514,0,1,0,1,0,0,1,1,0,0,1,114.7,139.1,-24.4,118.9,141.1,-22.2


In [114]:
test_df_regress.corr()[y_values]

,OFF_RATING,DEF_RATING,NET_RATING,E_OFF_RATING,E_DEF_RATING,E_NET_RATING
D. Bertans,0.088458,-0.545134,0.520709,0.062070,-0.560472,0.469887
D. Finney-Smith,-0.088458,0.545134,-0.520709,-0.062070,0.560472,-0.469887
D. Powell,0.019051,0.036323,-0.011983,0.004750,0.001722,0.002264
J. Brunson,0.173275,-0.225205,0.340548,0.180092,-0.200950,0.308992
J. Green,0.214119,-0.517488,0.612712,0.287378,-0.536245,0.653291
K. Porzingis,-0.354453,0.034039,-0.353914,-0.314971,0.061998,-0.326280
L. Doncic,0.279222,0.645789,-0.262427,0.267756,0.662464,-0.250465
M. Kleber,0.237337,0.158217,0.092512,0.239339,0.147411,0.104506
R. Bullock,-0.241429,-0.172861,-0.084274,-0.317095,-0.174134,-0.155016
S. Dinwiddie,0.231480,-0.187572,0.365373,0.152887,-0.257872,0.327194
